<a href="https://colab.research.google.com/github/shashaaankk/GradientAscent/blob/main/GradientAscent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# imports
import pandas as pd
import numpy as np
import gpxpy
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [12]:
LOCAL = True

In [13]:
if not LOCAL:
    !pip install --quiet kaggle kagglehub[pandas-datasets]
    from google.colab import files
    uploaded = files.upload()   # click to select your kaggle.json
    if 'kaggle.json' not in uploaded:
        raise FileNotFoundError("You must upload the kaggle.json you downloaded from Kaggle.")


In [14]:
if not LOCAL:
    import os, shutil
    # make sure ~/.kaggle exists
    kaggle_dir = os.path.expanduser("~/.kaggle")
    os.makedirs(kaggle_dir, exist_ok=True)

    # move and secure
    shutil.move("kaggle.json", os.path.join(kaggle_dir, "kaggle.json"))
    os.chmod(os.path.join(kaggle_dir, "kaggle.json"), 0o600)

    # sometimes needed:
    os.environ['KAGGLE_CONFIG_DIR'] = kaggle_dir


In [15]:
if not LOCAL:
    import kagglehub
    path = kagglehub.dataset_download("roccoli/gpx-hike-tracks")
else:
    !mkdir data
    !curl -L -o ./data/gpx-hike-tracks.zip https://www.kaggle.com/api/v1/datasets/download/roccoli/gpx-hike-tracks
    !unzip -o data/gpx-hike-tracks.zip -d data/

A subdirectory or file data already exists.


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

  0 57.4M    0  174k    0     0   185k      0  0:05:17 --:--:--  0:05:17  185k
 31 57.4M   31 18.0M    0     0  9189k      0  0:00:06  0:00:02  0:00:04 16.7M
 76 57.4M   76 44.1M    0     0  15.0M      0  0:00:03  0:00:02  0:00:01 22.1M
100 57.4M  100 57.4M    0     0  16.9M      0  0:00:03  0:00:03 --:--:-- 23.4M
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [16]:
# 1. Install gpxpy (to parse .gpx files)
import gpxpy
import pandas as pd
if not LOCAL:
    !pip install --quiet gpxpy

    # 2. Import libs
    import os, glob


    csv_files = glob.glob(os.path.join(path, "**", "*.csv"), recursive=True)
    csv_path = csv_files[0]
    print("Loading:", csv_path)

else:
    csv_path = "data/gpx-tracks-from-hikr.org.csv"

# Read and inspect
df = pd.read_csv(csv_path)
# print("Shape:", df.shape)
# print("Columns:", df.columns.tolist())
# print(df.head())



In [17]:
#Pre-processing 1

df = df.dropna()
# Convert time columns to datetime
df["start_time"] = pd.to_datetime(df["start_time"], format="%Y-%m-%d %H:%M:%S" , errors='coerce')
df["end_time"] = pd.to_datetime(df["end_time"], format="%Y-%m-%d %H:%M:%S" ,errors='coerce')

# Compute total duration in seconds
df["duration"] =  df["moving_time"]

# Compute break time: duration - moving_time
df["break_time"] = (df["end_time"] - df["start_time"]).dt.total_seconds() - df["moving_time"]

# Select relevant features
selected = df[["duration","length_3d", "min_elevation", "max_elevation", "break_time", "uphill", "downhill"]]

X = selected
y = df['difficulty'].str[1].astype(int)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train logistic regression
mask = X.notnull().all(axis=1) & y.notnull()
X_clean = X_scaled[mask]
y_clean = y[mask]


In [18]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(
        X_clean, y_clean, test_size=0.2, random_state=42, stratify=y_clean
    )

In [19]:
# Model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.59      0.44      0.51       157
           2       0.51      0.58      0.54       596
           3       0.43      0.70      0.53       693
           4       0.26      0.03      0.05       295
           5       0.67      0.01      0.03       152
           6       0.00      0.00      0.00        56

    accuracy                           0.47      1949
   macro avg       0.41      0.29      0.28      1949
weighted avg       0.45      0.47      0.41      1949



c:\Users\priyansh\anaconda3\envs\py_ml_313\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\priyansh\anaconda3\envs\py_ml_313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\priyansh\anaconda3\envs\py_ml_313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\priyansh\